In [2]:
#%pip install numpy
#%pip install Pandas

In [3]:
import numpy as np
import pandas as pd

In [4]:
import glob
from datetime import timedelta
results_files = glob.glob('Results_*.csv')
results_files

history = pd.concat([pd.read_csv(f) for f in results_files])
history = history[history['ResultStatus'] == 'CompletedRace']
history['Off'] =  pd.to_datetime(history['Off'], format='%m/%d/%Y %H:%M:%S')
history_start = history['Off'].min().date()
history_end = history['Off'].max().date() - timedelta(days=1)
predict_window_size = 90

In [5]:
from typing import Callable

def predict_races_using(prediction_func: Callable[[pd.DataFrame, pd.DataFrame], pd.DataFrame], max_days: int = -1) -> pd.DataFrame:
    window_start = history_start
    loop_end = history_end - timedelta(days=predict_window_size)   
    daily_predictions_made = 1
    prediction_summary = pd.DataFrame(columns=['Date', 'Races', 'Predicted', 'Wins', 'Losses', 'Gains'])
    while window_start < loop_end:
        window_end = window_start + timedelta(days=predict_window_size)
        window = history[(history['Off'].dt.date >= window_start) & (history['Off'].dt.date < window_end)].copy()
        prediction_start = window_end + timedelta(days=1)
        prediction_end = prediction_start + timedelta(days=1)
        race_cards = history[(history['Off'].dt.date >= prediction_start) & (history['Off'].dt.date < prediction_end)].copy()
        race_count = race_cards["RaceId"].nunique()
        
        predictions = prediction_func(race_cards, window)

        predicted = len(predictions)
        staked = predicted # £1 stake per prediction
        winners = predictions[predictions['PredictedPosition'] == predictions['FinishingPosition']]
        wins = len(winners)
        losses = predicted - wins
        percentageWins = (wins / predicted) * 100.0;
        winnings = winners['DecimalOdds'].sum()
        percentageGains = ((winnings - losses) / staked) * 100.0;
        print(f'Scored: {predicted}, Won: {wins}, Winnings (with £1 stake): {winnings}, Lost: {losses}, %gains/loss: {percentageGains}')

        row = pd.DataFrame([
            {
                'Date': prediction_start, 
                'Races': race_count, 
                'Predicted': predicted, 
                'Wins': wins, 
                'Winnings': winnings,
                'Losses': losses,
                'PercentageWins': percentageWins,
                'GainLoss': winnings - staked, 
                'PercentGainLoss': percentageGains
            }])
        prediction_summary = pd.concat([prediction_summary, row], axis=0, ignore_index=True)

        window_start = window_start + timedelta(days=1)
        daily_predictions_made = daily_predictions_made + 1
        if max_days != -1 and daily_predictions_made > max_days:
            break
    return prediction_summary

## First Runner Predictor
Baseline predictions by choosing the first horse on each race card.

Should be fairly random and allow us to score more **real** predictions against dumb luck.

In [6]:
def first_runner_predictor(race_cards : pd.DataFrame, prior_race_results: pd.DataFrame) -> pd.DataFrame:
    results = race_cards.sort_values('RaceCardNumber').groupby('RaceId').first().copy()
    results['PredictedPosition'] = 1
    return results

In [7]:
first_runner_predictions = predict_races_using(first_runner_predictor, max_days=10)
first_runner_predictions 

Scored: 52, Won: 13, Winnings (with £1 stake): 95.78205128205128, Lost: 39, %gains/loss: 109.19625246548324
Scored: 30, Won: 2, Winnings (with £1 stake): 5.7, Lost: 28, %gains/loss: -74.33333333333334
Scored: 58, Won: 9, Winnings (with £1 stake): 46.875757575757575, Lost: 49, %gains/loss: -3.6624869383490086
Scored: 51, Won: 8, Winnings (with £1 stake): 33.919444444444444, Lost: 43, %gains/loss: -17.805010893246187
Scored: 60, Won: 16, Winnings (with £1 stake): 90.93560606060606, Lost: 44, %gains/loss: 78.2260101010101
Scored: 90, Won: 16, Winnings (with £1 stake): 80.12575757575758, Lost: 74, %gains/loss: 6.806397306397306
Scored: 57, Won: 10, Winnings (with £1 stake): 31.749999999999996, Lost: 47, %gains/loss: -26.754385964912288
Scored: 38, Won: 6, Winnings (with £1 stake): 31.333333333333336, Lost: 32, %gains/loss: -1.7543859649122744
Scored: 42, Won: 9, Winnings (with £1 stake): 31.144444444444442, Lost: 33, %gains/loss: -4.417989417989424
Scored: 42, Won: 11, Winnings (with £1 st

,Date,Races,Predicted,Wins,Losses,Gains,Winnings,PercentageWins,GainLoss,PercentGainLoss
0,2022-06-06,52,52,13,39,NaN,95.782051,25.000000,43.782051,109.196252
1,2022-06-07,30,30,2,28,NaN,5.700000,6.666667,-24.300000,-74.333333
2,2022-06-08,58,58,9,49,NaN,46.875758,15.517241,-11.124242,-3.662487
3,2022-06-09,51,51,8,43,NaN,33.919444,15.686275,-17.080556,-17.805011
4,2022-06-10,60,60,16,44,NaN,90.935606,26.666667,30.935606,78.226010
5,2022-06-11,90,90,16,74,NaN,80.125758,17.777778,-9.874242,6.806397
6,2022-06-12,57,57,10,47,NaN,31.750000,17.543860,-25.250000,-26.754386
7,2022-06-13,38,38,6,32,NaN,31.333333,15.789474,-6.666667,-1.754386
8,2022-06-14,42,42,9,33,NaN,31.144444,21.428571,-10.855556,-4.417989
9,2022-06-15,42,42,11,31,NaN,83.948718,26.190476,41.948718,126.068376


In [8]:
first_runner_predictions.agg(
    {
        "PercentageWins" : ["average", "std"],
        "GainLoss": ["min", "max", "average", "skew", "std", "sum"],
        "Winnings": ["min", "max", "average", "skew", "std", "sum"],
    }
)

,PercentageWins,GainLoss,Winnings
average,18.826701,1.151511,53.151511
std,6.157394,26.923038,31.623393
min,NaN,-25.250000,5.700000
max,NaN,43.782051,95.782051
skew,NaN,0.892826,0.154603
sum,NaN,11.515113,531.515113


## Best rating predictor

Predict the winner of a race using the Official rating (note Racing post rating is not a useful predictor as it seems to be updated after the race and so is only really valid after the race)

In [9]:
def best_rating_runner_predictor(race_cards : pd.DataFrame, prior_race_results: pd.DataFrame) -> pd.DataFrame:
    results = race_cards.sort_values('OfficialRating', ascending=False).groupby('RaceId').first().copy()
    results['PredictedPosition'] = 1
    return results

In [10]:
best_rating_runner_predictions = predict_races_using(best_rating_runner_predictor, max_days=10)
best_rating_runner_predictions

Scored: 52, Won: 28, Winnings (with £1 stake): 185.94871794871793, Lost: 24, %gains/loss: 311.43984220907294
Scored: 30, Won: 10, Winnings (with £1 stake): 44.85, Lost: 20, %gains/loss: 82.83333333333334
Scored: 58, Won: 18, Winnings (with £1 stake): 99.52142857142859, Lost: 40, %gains/loss: 102.62315270935962
Scored: 51, Won: 23, Winnings (with £1 stake): 183.46944444444443, Lost: 28, %gains/loss: 304.84204793028323
Scored: 60, Won: 29, Winnings (with £1 stake): 213.5462121212121, Lost: 31, %gains/loss: 304.24368686868684
Scored: 90, Won: 50, Winnings (with £1 stake): 339.8424242424242, Lost: 40, %gains/loss: 333.15824915824913
Scored: 57, Won: 28, Winnings (with £1 stake): 172.6361111111111, Lost: 29, %gains/loss: 251.9931773879142
Scored: 38, Won: 15, Winnings (with £1 stake): 57.217424242424244, Lost: 23, %gains/loss: 90.04585326953747
Scored: 42, Won: 15, Winnings (with £1 stake): 65.66464646464647, Lost: 27, %gains/loss: 92.05868205868207
Scored: 42, Won: 20, Winnings (with £1 st

,Date,Races,Predicted,Wins,Losses,Gains,Winnings,PercentageWins,GainLoss,PercentGainLoss
0,2022-06-06,52,52,28,24,NaN,185.948718,53.846154,133.948718,311.439842
1,2022-06-07,30,30,10,20,NaN,44.850000,33.333333,14.850000,82.833333
2,2022-06-08,58,58,18,40,NaN,99.521429,31.034483,41.521429,102.623153
3,2022-06-09,51,51,23,28,NaN,183.469444,45.098039,132.469444,304.842048
4,2022-06-10,60,60,29,31,NaN,213.546212,48.333333,153.546212,304.243687
5,2022-06-11,90,90,50,40,NaN,339.842424,55.555556,249.842424,333.158249
6,2022-06-12,57,57,28,29,NaN,172.636111,49.122807,115.636111,251.993177
7,2022-06-13,38,38,15,23,NaN,57.217424,39.473684,19.217424,90.045853
8,2022-06-14,42,42,15,27,NaN,65.664646,35.714286,23.664646,92.058682
9,2022-06-15,42,42,20,22,NaN,138.273718,47.619048,96.273718,276.842186


In [11]:
best_rating_runner_predictions.agg(
    {
        "PercentageWins" : ["average", "std"],
        "GainLoss": ["min", "max", "average", "skew", "std", "sum"],
        "Winnings": ["min", "max", "average", "skew", "std", "sum"],
    }
)

,PercentageWins,GainLoss,Winnings
average,43.913072,98.097013,150.097013
std,8.568347,75.108858,89.731856
min,NaN,14.850000,44.850000
max,NaN,249.842424,339.842424
skew,NaN,0.686916,0.861515
sum,NaN,980.970127,1500.970127


## Fastest horse predictor

Predict the winner of a race by calculating the fastest horse in the race using the historic date on previous runs over the same race type (flat, hurdles, etc.), distance type (short, medium, long distance) and going (firm, soft, good etc.)

#### Classify distances by type

In [12]:
def calculate_distance_type(row):
    if row['DistanceInMeters'] < 1300:
        return 'VeryShort'
    elif row['DistanceInMeters'] < 1700:
        return 'Short'
    elif row['DistanceInMeters'] < 3000:
        return 'Medium'
    elif row['DistanceInMeters'] < 4000:        
        return 'Long'
    else:
        return 'VeryLong'

#### Calculate average speed over previous races

Sum the total distance of previous races and divide by total race time. This function will be used to aggregate results that have already been grouped by race type, distance type, and going. 

In [13]:
def calculate_average_speed_over_previous_races(x):
    d = {}
    d['RacesRan'] = x['HorseId'].count()
    d['TotalDistanceInMeters'] = x['DistanceInMeters'].sum()
    d['TotalTimeInSeconds'] = x['RaceTimeInSeconds'].sum()
    d['AverageSpeed'] = d['TotalDistanceInMeters'] / d['TotalTimeInSeconds']
    return pd.Series(d, index=['RacesRan', 'TotalDistanceInMeters', 'TotalTimeInSeconds', 'AverageSpeed'])

In [14]:
def calculate_speed_race_aggregates(x):
    d = {}
    d['HorseCount'] = x['HorseId'].count()
    d['PreviouslyRanOnSimilarCourseCount'] = x[x['AverageSpeed'] > 0]['HorseId'].count()
    return pd.Series(d, index=['HorseCount', 'PreviouslyRanOnSimilarCourseCount'])

In [15]:
def average_speed_predictor(race_cards : pd.DataFrame, prior_race_results: pd.DataFrame) -> pd.DataFrame:
    print('Calculating distance types...')
    prior_race_results['DistanceType'] = prior_race_results.apply(calculate_distance_type, axis=1)
    print(f'Calculated distance types for {len(prior_race_results)} history rows...')
    race_cards['DistanceType'] = race_cards.apply(calculate_distance_type, axis=1)
    print(f'Calculated distance types for {len(race_cards)} race cards rows...')
    
    print('Calculating average speeds...')    
    average_speeds = prior_race_results.groupby(['HorseId', 'RaceType', 'DistanceType', 'Going']).apply(calculate_average_speed_over_previous_races)
    print(f'Calculated average speeds for {len(average_speeds)} horses...')
    results_with_speeds = pd.merge(race_cards, average_speeds, how='left', on=['HorseId', 'RaceType', 'DistanceType', 'Going'])
    results_with_speeds['AverageSpeed'] = results_with_speeds['AverageSpeed'].fillna(0)

    # Filter out races that don't have "enough" horses with average speeds
    minimum_previous_runners = 1
    races_with_speed_counts = results_with_speeds.groupby('RaceId').apply(calculate_speed_race_aggregates)
    races_ids_to_predict = races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] > minimum_previous_runners].reset_index()['RaceId'].to_list()
    races_to_predict = results_with_speeds[results_with_speeds['RaceId'].isin(races_ids_to_predict) & results_with_speeds['AverageSpeed'] > 0]

    results = races_to_predict.sort_values('AverageSpeed', ascending=False).groupby('RaceId').first().copy()
    results['PredictedPosition'] = 1  
    return results    


In [16]:
average_speed_predictions = predict_races_using(average_speed_predictor, max_days=1)
average_speed_predictions

Calculating distance types...
Calculated distance types for 49776 history rows...
Calculated distance types for 478 race cards rows...
Calculating average speeds...
Calculated average speeds for 35756 horses...
Scored: 18, Won: 4, Winnings (with £1 stake): 44.83333333333333, Lost: 14, %gains/loss: 171.29629629629628


,Date,Races,Predicted,Wins,Losses,Gains,Winnings,PercentageWins,GainLoss,PercentGainLoss
0,2022-06-06,52,18,4,14,NaN,44.833333,22.222222,26.833333,171.296296
